<a href="https://colab.research.google.com/github/Mahnazshamissa/Back-Up-Befor-BackEnd/blob/master/asigmo_ex4_fatemeh_frankenstein_txt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exercise 4


Per group create a teams-meeting where one person opens the colab notebook and shares the screen (preferably a different person than yesterday)

1.   Change the data source from "frankenstein.txt" to "dracula.txt", 
2.   Train the network for 10 epochs and test it on the 3 example sentences
3.   Write down 3 more example sentences and test the model on these as well
4.   Train the LSTM for 50 more epochs and run the completion again. What do you observe?
4.   Add another LSTM layer to our RNN model
5.   Train the 2-layer LSTM for 10 epochs and see how the sentence completion has changed
6.   Change the "dracula.txt" to "trump.txt", which contains Tweets from Donald Trump from 2019 and 2020.
7.   Train a single-layer LSTM on the Tweet's from Trump. If you run out of memory while preprocessing the data, change ```step = 3``` to ```step = 10```.
8.   Run the completion on the six sentences from before again. What do you observe?



**Question**
The models make very few spelling errors and few grammar error, but the content is mostly nonsense. Why do you think are the error types so different?

We will meet in the general channel at 12:10!

In [1]:
! nvidia-smi

Fri Oct 30 23:24:53 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.32.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   55C    P8    10W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import tensorflow as tf
import numpy as np

# Downloading the data

In [3]:
filename = "frankenstein.txt" 
path = tf.keras.utils.get_file(
    filename, origin=f"https://pub.ist.ac.at/~mlechner/datasets/{filename}"
)
with open(path, encoding="utf-8") as f:
    text = f.read().lower()           # Make lowercase 
text = text.replace("\n", " ")        # Remove line-breaks & newlines
print("Corpus length:", len(text))

434176/430265 [==============================] - 1s 1us/step
Corpus length: 420726


In [4]:
# Print the first 100 characters of the file we downloaded
print("First 100 characters: ",text[:100])
print("Last 100 characters : ",text[-100:])

First 100 characters:  frankenstein, or, the modern prometheus by mary wollstonecraft (godwin) shelley  letter 1  _to mrs. 
Last 100 characters :  h lay close to the vessel.  he was soon borne away by the waves and lost in darkness and distance.  


# Define input features and output labels

In [5]:
chars = sorted(list(set(text)))
print("Total chars:", len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

maxlen = 50
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i : i + maxlen])
    next_chars.append(text[i + maxlen])
print("Number of sequences:", len(sentences))

Total chars: 59
Number of sequences: 140226


In [6]:
print("First sentence:")
print(sentences[0])
print("First label: ")
print(next_chars[0])

First sentence:
frankenstein, or, the modern prometheus by mary wo
First label: 
l


# Transforming text into numerical data

In [7]:
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros(len(sentences), dtype=np.int32)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i] = char_indices[next_chars[i]]

In [8]:
# First training samples
print(x[0])
print("label: ")
print(y[0])

[[False False False ... False False False]
 [False False False ... False False False]
 [False False False ... False False False]
 ...
 [ True False False ... False False False]
 [False False False ... False False False]
 [False False False ... False False False]]
label: 
34


# Splitting training and validation data

In [9]:
# We can define our training and validation dataset as simple numpy arrays
split_index = int(0.2*x.shape[0])
valid_x, valid_y = x[-split_index:],y[-split_index:]
train_x, train_y = x[:-split_index],y[:-split_index]
print("valid_x.shape",valid_x.shape)
print("valid_y.shape",valid_y.shape)
print("train_x.shape",train_x.shape)
print("train_y.shape",train_y.shape)

valid_x.shape (28045, 50, 59)
valid_y.shape (28045,)
train_x.shape (112181, 50, 59)
train_y.shape (112181,)


# Defining our Recurrent Neural Network

In [10]:
model = tf.keras.Sequential(
    [
        tf.keras.Input(shape=(maxlen, len(chars))),
        tf.keras.layers.LSTM(128),
        tf.keras.layers.Dense(len(chars), activation="softmax"),
    ]
)
optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.01)
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer,metrics=["sparse_categorical_accuracy"])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 128)               96256     
_________________________________________________________________
dense (Dense)                (None, 59)                7611      
Total params: 103,867
Trainable params: 103,867
Non-trainable params: 0
_________________________________________________________________


# Training

In [11]:
training_log = model.fit(train_x, train_y, batch_size=64, epochs=10, validation_data=(valid_x,valid_y))

Epoch 1/10
1753/1753 [==============================] - 11s 6ms/step - loss: 1.9761 - sparse_categorical_accuracy: 0.4172 - val_loss: 1.7010 - val_sparse_categorical_accuracy: 0.4883
Epoch 2/10
1753/1753 [==============================] - 10s 6ms/step - loss: 1.6157 - sparse_categorical_accuracy: 0.5174 - val_loss: 1.6018 - val_sparse_categorical_accuracy: 0.5248
Epoch 3/10
1753/1753 [==============================] - 10s 6ms/step - loss: 1.5089 - sparse_categorical_accuracy: 0.5501 - val_loss: 1.5679 - val_sparse_categorical_accuracy: 0.5346
Epoch 4/10
1753/1753 [==============================] - 10s 6ms/step - loss: 1.4542 - sparse_categorical_accuracy: 0.5655 - val_loss: 1.5356 - val_sparse_categorical_accuracy: 0.5436
Epoch 5/10
1753/1753 [==============================] - 10s 6ms/step - loss: 1.4173 - sparse_categorical_accuracy: 0.5742 - val_loss: 1.5282 - val_sparse_categorical_accuracy: 0.5535
Epoch 6/10
1753/1753 [==============================] - 10s 6ms/step - loss: 1.3907 -

# Let's try out our model

In [12]:
def sample(preds, temperature=0.5):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype("float64")
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)
    
def complete_sentence(sentence):
  if len(sentence) < maxlen:
    raise ValueError(f"Sentence must have at least {maxlen} characters, but provided sentence had only {sentence}")
  if len(sentence) > maxlen:
    sentence = sentence[-maxlen:]
  generated = ""
  print(f"Continuing sentence '{sentence}' ...")

  for i in range(200):
      x_pred = np.zeros((1, maxlen, len(chars)))
      for t, char in enumerate(sentence):
          x_pred[0, t, char_indices[char]] = 1.0
      preds = model.predict(x_pred, verbose=0)[0]
      next_index = sample(preds)
      next_char = indices_char[next_index]
      sentence = sentence[1:] + next_char
      generated += next_char

  print("... ", generated)
  print()

In [13]:
complete_sentence("this is an example to show how the machine would continue")

Continuing sentence ' an example to show how the machine would continue' ...
...  d of happiny and all the stat of the bearnes of the strent which my strangelf to make the mists of the studinb, and i felt that i am as a commence of the strenge with you could descended to the moon, 



In [20]:
### TODO: Add 3 more sentences to complete. Think about what types of sentences would be interesting to test

complete_sentence("this is an example to show how the machine would continue")
complete_sentence("I am a power woman and mother and i was a happy girl in my life ")
complete_sentence("i am a power girl because in my entire life i am happy and hopefull")

Continuing sentence ' an example to show how the machine would continue' ...
...  d in the promise of the work for his streng with the state of my unmaterable to all the strange and shatter and that he and in the charmanch of the fiend of the cour instruction to and the strenge wit

Continuing sentence 'oman and mother and i was a happy girl in my life ' ...
...  some my whole the presence of your inevise the father, which should have so england the moon to the scenes of the countenance of the mind to his present the protents of the beauty of describe relation

Continuing sentence ' because in my entire life i am happy and hopefull' ...
...  y to her who had describe to discover his state of selfming my sound of the thinks of the companio“s, and his secalied made the strection of the fathy of the living the studinbs of the since of the mo



# Biased data -> biased model

In [21]:
complete_sentence("    was the thiefs answer, to which the man replied ")
complete_sentence("    was the thiefs answer, to which the woman replied ")

Continuing sentence '  was the thiefs answer, to which the man replied ' ...
...  that i alone, and the scene of my food the scenes of the beauty of the lase, and the strenge more men had donmed in my their to my mind that i appeared to the labours of the seck with the proverdicant

Continuing sentence 'was the thiefs answer, to which the woman replied ' ...
...  my sky of the fullic and strowe with a become our death.  the rain of callestion of a seckinns, and at the more and unbranches.  about which i had desired to and the streins of the means of the more o

